In [31]:
from langchain.llms import GooglePalm
from dotenv import load_dotenv
import os
import google.generativeai as palm
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader

from langchain.prompts import PromptTemplate


In [3]:
load_dotenv()


True

In [4]:
llm=GooglePalm(google_api_key=os.environ['GOOGLE_API_KEY'],temperature=0.5)
result=llm("how is the weather")
print(result)

The weather is currently 51 degrees Fahrenheit with partly cloudy skies. There is a 30% chance of rain today.


In [5]:
embeddings=HuggingFaceEmbeddings()
good=embeddings.embed_query("This is a test document.")
print(len(good))

768


In [7]:

loader = CSVLoader(file_path="FAQ.csv")
data = loader.load()
data

[Document(page_content='Prompt: How do I exchange my existing commercial vehicle?\nanswer: To exchange your commercial vehicle click on the Exchange button in the Navigational Menu Bar. \r\nClicking on the exchange button leads to the TATA OK page which helps in buying, \r\nselling or exchanging commercial vehicles of any make, model or vintage.', metadata={'source': 'C:\\Users\\vetri\\Downloads\\FAQ.csv', 'row': 0}),
 Document(page_content='Prompt: How do I get to know the final offer price after I have booked my vehicle?\nanswer: The booking amount paid by you Is used to simply reserve the vehicle in your name. The final offer price is communicated by the dealer.', metadata={'source': 'C:\\Users\\vetri\\Downloads\\FAQ.csv', 'row': 1}),
 Document(page_content='Prompt: How do I know my booking has been confirmed?\nanswer: Once you have completed the payment, you will receive a Thank You message confirming your payment. \r\nThis denotes that your preferred commercial vehicle has been bo

In [20]:
VS=FAISS.from_documents(documents=data,embedding=embeddings)
VS

In [25]:
retriever =VS.as_retriever()
ret_data= retriever.get_relevant_documents ("profit in 2020")
ret_data

[Document(page_content='Prompt: what is the profit in 2020\nanswer: 30 thousand core', metadata={'source': 'C:\\Users\\vetri\\Downloads\\FAQ.csv', 'row': 10}),
 Document(page_content='Prompt: when the sales was down\nanswer: 2020', metadata={'source': 'C:\\Users\\vetri\\Downloads\\FAQ.csv', 'row': 9}),
 Document(page_content='Prompt: which year the sell of cars were high\nanswer: 2019', metadata={'source': 'C:\\Users\\vetri\\Downloads\\FAQ.csv', 'row': 8}),
 Document(page_content='Prompt: how much of workers works in the company\nanswer: around 20000 people', metadata={'source': 'C:\\Users\\vetri\\Downloads\\FAQ.csv', 'row': 11})]

In [9]:
from langchain.chains import RetrievalQA

In [26]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    input_key="query",
    retriever=retriever,
    return_source_documents=True
)

In [30]:
qa("profit in 2020")

{'query': 'profit in 2020',
 'result': '30 thousand core',
 'source_documents': [Document(page_content='Prompt: what is the profit in 2020\nanswer: 30 thousand core', metadata={'source': 'C:\\Users\\vetri\\Downloads\\FAQ.csv', 'row': 10}),
  Document(page_content='Prompt: when the sales was down\nanswer: 2020', metadata={'source': 'C:\\Users\\vetri\\Downloads\\FAQ.csv', 'row': 9}),
  Document(page_content='Prompt: which year the sell of cars were high\nanswer: 2019', metadata={'source': 'C:\\Users\\vetri\\Downloads\\FAQ.csv', 'row': 8}),
  Document(page_content='Prompt: how much of workers works in the company\nanswer: around 20000 people', metadata={'source': 'C:\\Users\\vetri\\Downloads\\FAQ.csv', 'row': 11})]}

In [42]:

prompt_temp="""Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
if the answer is not found in the context, must reply "This Question not Present in My Database." Don't try to make up an answer.
CONTEXT: {context}
QUESTION: {question}"""
PROMPT = PromptTemplate(template=prompt_temp, input_variables=["context", "question"])

In [53]:
rh=qa("who is Vijay")
rh

{'query': 'who is Vijay',
 'result': 'CEO, Tata Motors',
 'source_documents': [Document(page_content='Prompt: who is the programmer of the web portal\nanswer: Unknown coder', metadata={'source': 'C:\\Users\\vetri\\Downloads\\FAQ.csv', 'row': 7}),
  Document(page_content='Prompt: How do I exchange my existing commercial vehicle?\nanswer: To exchange your commercial vehicle click on the Exchange button in the Navigational Menu Bar. \r\nClicking on the exchange button leads to the TATA OK page which helps in buying, \r\nselling or exchanging commercial vehicles of any make, model or vintage.', metadata={'source': 'C:\\Users\\vetri\\Downloads\\FAQ.csv', 'row': 0}),
  Document(page_content='Prompt: What is the minimum price of the car\nanswer: ten lakh', metadata={'source': 'C:\\Users\\vetri\\Downloads\\FAQ.csv', 'row': 6}),
  Document(page_content='Prompt: How do I know my booking has been confirmed?\nanswer: Once you have completed the payment, you will receive a Thank You message confirm

In [55]:
rh["result"]

'CEO, Tata Motors'